<a href="https://colab.research.google.com/github/onehungrybird/Agentic_AI_travel_planner/blob/main/tool_using_AI_Agent2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y langchain langchain-core langchain-community

Found existing installation: langchain 0.3.21
Uninstalling langchain-0.3.21:
  Successfully uninstalled langchain-0.3.21
Found existing installation: langchain-core 0.3.49
Uninstalling langchain-core-0.3.49:
  Successfully uninstalled langchain-core-0.3.49
Found existing installation: langchain-community 0.3.20
Uninstalling langchain-community-0.3.20:
  Successfully uninstalled langchain-community-0.3.20


In [1]:
!pip install --upgrade langchain langchain-core mistralai langchain-community ollama torch
!pip install transformers accelerate bitsandbytes --quiet
!pip install -U bitsandbytes

In [2]:
import warnings
from transformers import logging
logging.set_verbosity_error()
warnings.filterwarnings("ignore")

In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
weather_api = userdata.get('WEATHER_API')

## Define Open-Source Weather Tool

In [4]:
import requests
from langchain.tools import Tool

weather_api = weather_api
url = "http://api.openweathermap.org/data/2.5/weather"

def real_weather_fetcher(location):
    """fetches real-time weather from openweathermap API."""
    params = {"q": location, "appid": weather_api, "units": "metric"}
    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code == 200:
        temp = data["main"]["temp"]
        description = data["weather"][0]["description"]
        return f"{description}, {temp}°C"  # Simplified output
    return "Weather data unavailable"

weather_tool = Tool(
    name = "Real Weather API",
    func = real_weather_fetcher,
    description = "Fetches real-time weather for a given location"
)

## Adding more open source tool

In [5]:
def math_solver(expression):
  try:
    return eval(expression)
  except Exception as e:
    return str(e)

math_tool = Tool(
    name = 'Math solver',
    func = math_solver,
    description = "solves math expression"
)

def convert_to_fahrenheit(celsius_str):
    try:
        celsius = float(celsius_str.replace("°C", "").strip())
        return f"{(celsius * 9/5) + 32:.1f}°F"  # Cleaner format
    except ValueError:
        return "Invalid temperature"

converter_tool = Tool(
    name = "Unit converter",
    func = convert_to_fahrenheit,
    description = "Converts temperature from Celsius to Fahrenheit"
)

In [6]:
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
import re

class CleanOutputParser(AgentOutputParser):
    def parse(self, text: str):
        if "Final Answer:" in text:
            return AgentFinish(
                {"output": text.split("Final Answer:")[-1].split("For troubleshooting")[0].strip()},
                text
            )
        # Handle cases where LLM directly gives the answer
        weather_pattern = r"(?:rain|sun|cloud).*?\d+°[CF]"
        if re.search(weather_pattern, text, re.IGNORECASE):
            return AgentFinish({"output": text.strip()}, text)
        return AgentFinish({"output": text.strip()}, text)

In [7]:
import torch
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = quant_config,
    device_map = 'auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
text_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline = text_pipeline)

<ipython-input-8-a3165e67cdc3>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = text_pipeline)


In [9]:
from langchain.agents import initialize_agent, AgentType, AgentExecutor

tools = [math_tool, converter_tool, weather_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    max_iterations=2,
    early_stopping_method="generate",
    agent_kwargs={"output_parser": CleanOutputParser()},
    handle_parsing_errors="Check your output and try again"
)

<ipython-input-9-e0d29b7ea233>:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [10]:
try:
    response = agent.run("What's the weather in Madhubani?")
    # Clean the output
    final_answer = response.split("Observation:")[-1].strip()
    final_answer = final_answer.split("For troubleshooting")[0].strip()
    print(final_answer)
except Exception as e:
    print(f"Error: {str(e)}")
    # Try to extract any useful information from the error
    if "Final Answer:" in str(e):
        print(str(e).split("Final Answer:")[-1].split("For troubleshooting")[0].strip())

<ipython-input-10-31721cebeb92>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What's the weather in Madhubani?")


The current weather in Madhubani is light rain with a temperature of 90.5 degrees Fahrenheit.


In [13]:
response = agent.invoke({"input": "What is the weather in Delhi?"})
print(response['output'])

The current temperature in Delhi is 96 degrees Fahrenheit.


In [18]:
response = agent.invoke({"input": "Find the weather in Darbhanga and convert it to celcius."})
print(response['output'])

32.5 degrees Celsius


In [23]:
response = agent.invoke({"input": "what is 16*6"})
print(response['output'])

96


In [24]:
response = agent.invoke({"input": "Find the weather in Bhabua and convert it to celcius."})
print(response['output'])

35.6 degrees Celsius
